In [2]:
from base64 import encode
import spacy
from spacy.matcher import Matcher
import hashlib
import json
import numpy as np
from sklearn.metrics import precision_recall_fscore_support


nlp = spacy.load("en_core_web_sm")


/home/simret/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def expand_working_list(pat):
    result = []
    if (pat == None):
        return []
    else:
        working_list = []
        optionals = []
        combinations = pat.split("+")
        for pattern in combinations:
            patterns_within = pattern.split("|")
            if (len(patterns_within) > 1):
                # result.append(working_list)
                # working_list = []
                optional_patterns = []
                for p in patterns_within:
                    if (p[0] == "["):
                        temp = {"LEMMA": {"IN": [p[1:-1]]}, "OP": "+"}
                        optional_patterns.append(temp)
                    elif (p[0] == "$"):
                        temp = {"ENT_TYPE": p[1:], "OP": "+"}
                        optional_patterns.append(temp)
                    else:
                        temp = {"POS": p, "OP": "+"}
                        optional_patterns.append(temp)
                count = len(working_list)
                if (count):
                    while (count):
                        count -= 1
                        temp = working_list.pop(0)
                        for opt in optional_patterns:
                            updated_pattern = temp + [opt]
                            working_list.append(updated_pattern)
                else:
                    working_list = [[x] for x in optional_patterns]

                # result.append(optional_patterns)
            else:
                if (pattern == "*"):
                    temp = {"OP": "?"}
                    # temp = {"OP": "?"}
                elif (pattern[0] == "["):
                    temp = {"LEMMA": {"IN": [pattern[1:-1]]}, "OP": "+"}
                elif (pattern[0] == "$"):
                    temp = {"ENT_TYPE": pattern[1:], "OP": "+"}
                else:
                    temp = {"POS": pattern, "OP": "+"}
                if (len(working_list) == 0):
                    working_list.append([temp])
                else:
                    for i in range(len(working_list)):
                        working_list[i].append(temp)
        return working_list

In [59]:
from functools import reduce

def get_spanning(matches):
    ranges = [(x[1],x[2]) for x in matches]
    reducer = (lambda acc, el: acc[:-1:] + [(min(*acc[-1], *el), max(*acc[-1], *el))]
    if acc[-1][1] > el[0] else acc + [el] )
    spanning = reduce(reducer, ranges[1::], [ranges[0]])
    
    result_matches = []
    for i, j in spanning:
        result_matches.append([sent.split(" ")[i:j], i, j])
    return result_matches



In [60]:

def check_matching(sent, working_list, explain=False):
    matcher = Matcher(nlp.vocab)
    for index, patterns in enumerate(working_list):
        matcher.add(f"rule{index}", [patterns])
    doc = nlp(sent)
    
    matches = matcher(doc)
    if(matches is not None and len(matches)>0):
        if(explain):
            return(True, get_spanning(matches) )
        for id, start, end in matches:
            if(str(doc[start:end]).strip() !=""):
                return True
    if(explain):
        return (False, "")
    return False

In [61]:
sent = "I purchased $ 200 a Gibeon Meteorite and black zirconium wedding band from Rings Unique for approximately $ 1200 ."
pattern = "$MONEY"

In [62]:
def tokenizer(sent):
    s = nlp(sent)
    new_sent = ""
    for i in s:
        new_sent +=f"{i} "
    return new_sent

In [63]:
working_list = expand_working_list(pattern)

In [68]:

mm =check_matching(sent, working_list, explain=True)


In [69]:
mm

(True, [[['200'], 3, 4], [['approximately', '$', '1200'], 16, 19]])